In [ ]:
import numpy as np
from optic.models.devices import mzm, photodiode
from optic.models.channels import linearFiberChannel
from optic.comm.sources import bitSource
from optic.comm.modulation import modulateGray
from optic.comm.metrics import bert
from optic.dsp.core import firFilter, pulseShape, upsample, pnorm, anorm
from optic.utils import parameters, dBm2W
from scipy.special import erfc
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, Model


2026-02-13 13:42:56.476533: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-13 13:42:56.477878: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-13 13:42:56.550311: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-13 13:42:58.611889: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

In [4]:

def build_nn_dpd_model(input_shape=(None, 1)):
    """
    Implements the NN-based DPD architecture from the paper.
    Sections: (A) 101-tap CNN, (B) FFNN Core with ResNet, (C) 301-tap CNN.
    """
    # Input is the sequence of 2-PAM symbols x[n]
    inputs = layers.Input(shape=input_shape)

    # --- Section (A): Linear Memory (101 taps) ---
    # Equivalent to a linear FIR filter [cite: 260, 317]
    # Initialized as an 'Impulse' to start as a pass-through [cite: 250, 318]
    impulse_init_a = tf.constant_initializer(np.eye(101, 1, k=50)) # Center tap = 1
    section_a = layers.Conv1D(filters=1, kernel_size=101, padding='same', 
                              use_bias=True, kernel_initializer=impulse_init_a,
                              name="Section_A_101_taps")(inputs)

    # --- Section (B): Nonlinear Core (FFNN) ---
    # First layer is a short convolution to mix memory/nonlinearity [cite: 232, 261]
    # Then fully connected layers with Leaky ReLU [cite: 231, 262]
    # Layer 1: 11-tap conv -> 21 neurons [cite: 250, 261]
    b_conv = layers.Conv1D(filters=21, kernel_size=11, padding='same', 
                           activation=layers.LeakyReLU(alpha=0.1))(section_a)
    
    # Flattening for dense layers (processing symbol by symbol)
    # Layer 2: 12 neurons [cite: 250, 420]
    b_dense1 = layers.Dense(12, activation=layers.LeakyReLU(alpha=0.1))(b_conv)
    # Layer 3: 8 neurons [cite: 250, 420]
    b_dense2 = layers.Dense(8, activation=layers.LeakyReLU(alpha=0.1))(b_dense1)
    # Layer 4: 8 neurons [cite: 250, 420]
    b_dense3 = layers.Dense(8, activation=layers.LeakyReLU(alpha=0.1))(b_dense2)
    # Layer 5: 1 neuron (Linear) [cite: 250, 263]
    section_b_output = layers.Dense(1, activation='linear')(b_dense3)

    # --- ResNet Shortcut Bypass ---
    # Adds Section A output directly to Section B output [cite: 258, 416]
    resnet_add = layers.Add()([section_a, section_b_output])

    # --- Section (C): Linear Memory (301 taps) ---
    # Handles long-delay signal reflections 
    # Initialized as an 'Impulse' [cite: 250, 318]
    impulse_init_c = tf.constant_initializer(np.eye(301, 1, k=150))
    section_c = layers.Conv1D(filters=1, kernel_size=301, padding='same', 
                              use_bias=True, kernel_initializer=impulse_init_c,
                              name="Section_C_301_taps")(resnet_add)

    return Model(inputs=inputs, outputs=section_c, name="NN_DPD_Sandwich")

# Build the model
dpd_model = build_nn_dpd_model()
dpd_model.summary()

2026-02-13 13:43:01.928685: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/abood/major_proj/.venv/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "NN_DPD_Sandwich"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None, 1)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Section_A_101_taps  │ (None, None, 1)   │        102 │ input_layer[0][0] │
│ (Conv1D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, None, 21)  │        252 │ Section_A_101_ta… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None, 12)  │        264 │ conv1d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, None, 8)   │        104 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, None, 8)   │         72 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, None, 1)   │          9 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, None, 1)   │          0 │ Section_A_101_ta… │
│                     │                   │            │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Section_C_301_taps  │ (None, None, 1)   │        302 │ add[0][0]         │
│ (Conv1D)            │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,105 (4.32 KB)

 Trainable params: 1,105 (4.32 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# simulation parameters
SpS = 16  # samples per symbol
M = 2  # order of the modulation format
Rs = 10e9  # Symbol rate
Fs = SpS * Rs  # Signal sampling frequency (samples/second)
Pi_dBm = 3  # laser optical power at the input of the MZM in dBm
Pi = dBm2W(Pi_dBm)  # convert from dBm to W

# Bit source parameters
paramBits = parameters()
paramBits.nBits = 100000  # number of bits to be generated
paramBits.mode = 'random' # mode of the bit source 
paramBits.seed = 123      # seed for the random number generator

# pulse shaping parameters
paramPulse = parameters()
paramPulse.pulseType = 'nrz'  # pulse shape type
paramPulse.SpS = SpS     # samples per symbol  

# MZM parameters
paramMZM = parameters()
paramMZM.Vpi = 2
paramMZM.Vb = -paramMZM.Vpi / 2

# linear fiber optical channel parameters
paramCh = parameters()
paramCh.L = 100        # total link distance [km]
paramCh.alpha = 0.2    # fiber loss parameter [dB/km]
paramCh.D = 16         # fiber dispersion parameter [ps/nm/km]
paramCh.Fc = 193.1e12  # central optical frequency [Hz]
paramCh.Fs = Fs

# photodiode parameters
paramPD = parameters()
paramPD.ideal = False
paramPD.B = Rs
paramPD.Fs = Fs
paramPD.seed = 456  # seed for the random number generator



## Simulation
print("\nStarting simulation...", end="")

# generate pseudo-random bit sequence
bitsTx = bitSource(paramBits)

# generate 2-PAM modulated symbol sequence
symbTx = modulateGray(bitsTx, M, "pam")



# # --- NN-DPD APPLICATION STEP ---

# # 1. Reshape for TensorFlow: (1, Number_of_Symbols, 1)
# # TF Conv1D expects [Batch, Time, Features]
# nn_input = symbTx.reshape(1, -1, 1).astype(np.float32)

# # 2. Run Inference
# # We use the model to "pre-distort" the symbols
# # .predict() or calling the model directly returns the z[n] sequence
# z_n = dpd_model(nn_input, training=False)

# # 3. Convert back to a flat NumPy array for the optic library
# # We remove the extra dimensions added for TF
# symbTx_dpd = z_n.numpy().flatten()

# # 4. Continue with the standard simulation using the NEW symbols
# symbolsUp = upsample(symbTx_dpd, SpS)
# # ... the rest of your transmitter code (pulseShape, firFilter, mzm)


# upsampling
symbolsUp = upsample(symbTx, SpS)

# pulse shaping
pulse = pulseShape(paramPulse)
sigTx = firFilter(pulse, symbolsUp)
sigTx = anorm(sigTx) # normalize to 1 Vpp

# optical modulation
Ai = np.sqrt(Pi)  # ideal cw laser constant envelope
sigTxo = mzm(Ai, sigTx, paramMZM)

# linear fiber channel model
sigCh = linearFiberChannel(sigTxo, paramCh)

# noisy PD (thermal noise + shot noise + bandwidth limit)
I_Rx = photodiode(sigCh, paramPD)

# capture samples in the middle of signaling intervals
I_Rx = I_Rx[0::SpS]

print("simulation completed.")


# calculate the BER and Q-factor
BER, Q = bert(I_Rx)

print("\nTransmission performance metrics:")
print(f"Q-factor = {Q:.2f} ")
print(f"BER = {BER:.2e}")

# theoretical error probability from Q-factor
Pb = 0.5 * erfc(Q / np.sqrt(2))
print(f"Pb = {Pb:.2e}\n")


Starting simulation...simulation completed.

Transmission performance metrics:
Q-factor = 3.58 
BER = 1.00e-04
Pb = 1.73e-04



In [10]:
import numpy as np
import tensorflow as tf
from optic.models.devices import mzm, photodiode
from optic.models.channels import linearFiberChannel
from optic.comm.sources import bitSource
from optic.comm.modulation import modulateGray
from optic.comm.metrics import bert
from optic.dsp.core import firFilter, pulseShape, upsample, anorm
from optic.utils import parameters, dBm2W
from scipy.special import erfc

# --- [Assumes dpd_model is already defined using the build function provided earlier] ---

# 1. Setup Simulation Parameters
num_iterations = 9  # Based on paper convergence (Fig. 6) [cite: 370]
n_bits = 100000
SpS = 16
M = 2
Rs = 10e9
Fs = SpS * Rs
Pi_dBm = 3

# Define simulation objects
paramBits = parameters(); paramBits.nBits = n_bits; paramBits.mode = 'random'; paramBits.seed = 123
paramPulse = parameters(); paramPulse.pulseType = 'nrz'; paramPulse.SpS = SpS
paramMZM = parameters(); paramMZM.Vpi = 2; paramMZM.Vb = -1
paramCh = parameters(); paramCh.L = 80; paramCh.alpha = 0.2; paramCh.D = 16; paramCh.Fs = Fs
paramPD = parameters(); paramPD.ideal = False; paramPD.B = Rs; paramPD.Fs = Fs

# Initialize results tracking
results_ber = []

print("\nStarting ILA Training Loop...")

for i in range(num_iterations):
    # --- TRANSMITTER ---
    bitsTx = bitSource(paramBits)
    symbTx = modulateGray(bitsTx, M, "pam")
    
    # Apply DPD at Transmitter (x -> z)
    nn_input = symbTx.reshape(1, -1, 1).astype(np.float32)
    z_n = dpd_model(nn_input, training=False)
    symbTx_dpd = z_n.numpy().flatten()
    
    # Pulse shaping and Modulation
    symbolsUp = upsample(symbTx_dpd, SpS)
    pulse = pulseShape(paramPulse)
    sigTx = firFilter(pulse, symbolsUp)
    sigTx = anorm(sigTx)
    
    sigTxo = mzm(np.sqrt(dBm2W(Pi_dBm)), sigTx, paramMZM)
    
    # --- CHANNEL ---
    sigCh = linearFiberChannel(sigTxo, paramCh)
    
    # --- RECEIVER ---
    I_Rx_raw = photodiode(sigCh, paramPD)
    I_Rx = I_Rx_raw[0::SpS] # Downsample
    
    # --- PERFORMANCE TRACKING ---
    BER, Q = bert(I_Rx)
    results_ber.append(BER)
    print(f"Iteration {i+1}: BER = {BER:.2e}")
    
    # --- ILA TRAINING BLOCK ---
    # Target: The pre-distorted sequence we sent (z) [cite: 328]
    # Input: The messy sequence we received (y) [cite: 328]
    z_target = symbTx_dpd.reshape(1, -1, 1).astype(np.float32)
    y_input = (I_Rx / np.std(I_Rx)).reshape(1, -1, 1).astype(np.float32)
    
    # Training hyperparameters from Table II [cite: 293]
    dpd_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    dpd_model.fit(y_input, z_target, epochs=30, verbose=0) # [cite: 293]

print("\nILA Training Completed.")


Starting ILA Training Loop...
Iteration 1: BER = 5.02e-01
Iteration 2: BER = 4.98e-01
Iteration 3: BER = 3.00e-05
Iteration 4: BER = 3.00e-05
Iteration 5: BER = 4.98e-01
Iteration 6: BER = 5.00e-01
Iteration 7: BER = 3.00e-05
Iteration 8: BER = 5.01e-01
Iteration 9: BER = 3.00e-05

ILA Training Completed.
